In [1]:
from numpy import *  #科学计算包numpy
import numpy as np
import operator      #运算符模块
import sys
#k-近邻算法
# sys.path.append('/home/jimmyli/mlpractice/机器学习实战/')
#创建数据集
def createDataSet():
    group=array([[1.0,1.1],[1.0,1.0],[0,0],[0,0.1]])
    labels=['A','A','B','B']
    return group,labels

#计算距离
def classify0(inX,dataSet,labels,k):
    dataSetSize=dataSet.shape[0]   #shape读取数据矩阵第一维度的长度
    diffMat = tile(inX, (dataSetSize, 1)) - dataSet  #tile重复数组inX，有dataSet行 1个dataSet列，减法计算差值
    sqDiffMat=diffMat**2 #**是幂运算的""意思，这里用的欧式距离
    sqDisttances=sqDiffMat.sum(axis=1) #普通sum默认参数为axis=0为普通相加，axis=1为一行的行向量相加
    distances=sqDisttances**0.5
    sortedDistIndicies=distances.argsort() #argsort返回数值从小到大的索引值（数组索引0,1,2,3）
 #选择距离最小的k个点
    classCount={}
    for i in range(k):
        voteIlabel=labels[sortedDistIndicies[i]] #根据排序结果的索引值返回靠近的前k个标签
        classCount[voteIlabel]=classCount.get(voteIlabel,0)+1 #各个标签出现频率
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True) #排序频率
    #!!!!!  classCount.iteritems()修改为classCount.items()
    #sorted(iterable, cmp=None, key=None, reverse=False) --> new sorted list。
    # reverse默认升序 key关键字排序itemgetter（1）按照第一维度排序(0,1,2,3)
    return sortedClassCount[0][0]  #找出频率最高的

In [2]:
def file2matrix(filename):
    fr = open(filename)
    arrayOfLines = fr.readlines()
    numberOfLines = len(arrayOfLines)
    returnMat = np.zeros((numberOfLines, 3))
    classLabelVector = []
    index = 0
    for line in arrayOfLines:
        line = line.strip()
        listFromLine = line.split('\t')
        returnMat[index,:] = listFromLine[0:3]
        classLabelVector.append(int(listFromLine[-1]))
        index += 1
    return returnMat,classLabelVector

In [3]:
datingDataMat, datingLabels = file2matrix('/home/jimmyli/mlpractice/机器学习实战/ch2/datingTestSet2.txt')

In [4]:
import matplotlib
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(datingDataMat[:,1], datingDataMat[:,2],
          15.0*array(datingLabels), 15.0*array(datingLabels))
plt.show()

<Figure size 640x480 with 1 Axes>

In [5]:
def autoNorm(dataSet):
    minVals = dataSet.min(0)
    maxVlas = dataSet.max(0)
    ranges = maxVlas - minVals
#     normDataSet = np.zeros(shape(dataSet))
#     m = dataSet.shape[0]
#     normDataSet = normDataSet/tile(ranges, (m,1))
    normDataSet = dataSet - minVals
    normDataSet = normDataSet/ranges
    return normDataSet, ranges, minVals
normMat, ranges, minVals = autoNorm(datingDataMat)

In [6]:
normMat, ranges, minVals = autoNorm(datingDataMat)

In [7]:
normMat

array([[0.44832535, 0.39805139, 0.56233353],
       [0.15873259, 0.34195467, 0.98724416],
       [0.28542943, 0.06892523, 0.47449629],
       ...,
       [0.29115949, 0.50910294, 0.51079493],
       [0.52711097, 0.43665451, 0.4290048 ],
       [0.47940793, 0.3768091 , 0.78571804]])

In [8]:
def datingClassTest():
    hoRatio = 0.10
    datingDataMat, datingLabels = file2matrix('/home/jimmyli/mlpractice/机器学习实战/ch2/datingTestSet2.txt')
    normMat, ranges, minVals = autoNorm(datingDataMat)
    m = normMat.shape[0]
    numTestVecs = int(m*hoRatio)
    errorCount = 0.0
    for i in range(numTestVecs):
        classifierResult = classify0(normMat[i,:],normMat[numTestVecs:m,:],
                                    datingLabels[numTestVecs:m],3)
        print("the classifier came back with: %d, the real answer is: %d"
             % (classifierResult, datingLabels[i]))
        if (classifierResult != datingLabels[i]): errorCount += 1.0
    print ("the total error rate is: %f" % (errorCount/float(numTestVecs)))

In [13]:
tmp_category = '/home/jimmyli/mlpractice/机器学习实战/'
def classifyPerson():
    resultList = ['not at all', 'in small doess', 'in large doses']
    percentTats = float(input(\
                        "percentage of time spent palying video games?"))
    ffMiles = float(input("frequent flier miles earned per year?"))
    iceCream = float(input("liters of ice cream consumed per year?"))
    datingDataMat, datingLabels = file2matrix(tmp_category + 'ch2/datingTestSet2.txt')
    normMat, ranges, minVals = autoNorm(datingDataMat)
    inArr = np.array([ffMiles, percentTats, iceCream])
    classifierResult = classify0((inArr-minVals)/ranges,normMat, datingLabels, 3)
    print("You will probably like this person:",
         resultList[classifierResult - 1])
if __name__ == '__main__':
    classifyPerson()

percentage of time spent palying video games?10
frequent flier miles earned per year?10000
liters of ice cream consumed per year?0.5
You will probably like this person: in small doess


In [23]:
classifyPerson

<function __main__.classifyPerson()>

#### 手写识别系统

In [14]:
def img2vector(filename):
    returnVect = np.zeros((1, 1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0, 32*i+j] = int(lineStr[j])
    return returnVect

In [15]:
testVector = img2vector(tmp_category + 'ch2/testDigits/0_13.txt')

In [16]:
testVector[0,0:31]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [22]:
from os import listdir
def handwritingClassTest():
    hwLabels = []
    trainingFileList = listdir(tmp_category + 'ch2/trainingDigits')
    m = len(trainingFileList)
    trainingMat = zeros((m, 1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        hwLabels.append(classNumStr)
        trainingMat[i,:] = img2vector(tmp_category + 'ch2/trainingDigits/%s' % fileNameStr)
    testFileList = listdir('ch2/testDigits')
    errorCount = 0.0
    mTest = len(testFileList)
    for i in range(mTest):
        fileNameStr = testFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        vectorUnderTest = img2vector('ch2/testDigits/%s' % fileNameStr)
        classifierResult = classify0(vectorUnderTest, trainingMat,
                                    hwLabels, 3)
#         print('the classifier came back with: %d, the real answer\
#         is : %d' % (classifierResult, classNumStr))
        if (classifierResult != classNumStr): errorCount += 1.0
    print('\n the total number of errors is: %d' % errorCount)
    print('\n the total error rate is: %f' % (errorCount/float(mTest)))

In [23]:
handwritingClassTest()


 the total number of errors is: 11

 the total error rate is: 0.011628
